In [ ]:
import shelve
import pickle
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
try:
    import orjson as _json
    loads = _json.loads
except Exception:
    import json as _json
    loads = _json.loads

corpus = '../Inverted_index/trec-tot-2025-corpus.jsonl.gz'


In [5]:
import math 
def AND(l1, l2):
    # note: this method assumes that both lists are sorted and do not contain any duplicates (which if the input comes from a posting list is given)
    p1 = 0
    p2 = 0
    sqrtJump1 = max(math.isqrt(len(l1)), 1)
    sqrtJump2 = max(math.isqrt(len(l2)), 1)
    result = []
    while p1 < len(l1)  and p2 < len(l2):
        if l1[p1] == l2[p2]:
            result.append(l1[p1])
            p1 += 1
            p2 += 1
        # skip pointer in first list
        elif p1 % sqrtJump1 == 0 and p1 + sqrtJump1 < len(l1) and l1[p1 + sqrtJump1] <= l2[p2]:
            p1 += sqrtJump1
        # skip pointer in second list
        elif p2 % sqrtJump2 == 0 and p2 + sqrtJump2 < len(l2) and l2[p2 + sqrtJump2] <= l1[p1]:
            p2 += sqrtJump2
        elif l1[p1] < l2[p2]:
            p1 += 1
        else:
            p2 += 1
    return result

In [6]:
def OR(l1, l2):
    # note: this method assumes that both lists are sorted and do not contain any duplicates (which if the input comes from a posting list is given)
    p1 = 0
    p2 = 0
    result = []
    while p1 < len(l1) and p2 < len(l2):
        if l1[p1] < l2[p2]:
            result.append(l1[p1])
            p1 += 1
        elif l1[p1] > l2[p2]:
            result.append(l2[p2])
            p2 += 1
        else: 
            result.append(l1[p1])
            p1 += 1
            p2 += 1
    while p1 < len(l1):
        result.append(l1[p1])
        p1 += 1
    while p2 < len(l2):
        result.append(l2[p2])
        p2 += 1
    return result

In [8]:
#for stopword removal
stop = set(stopwords.words('english') + list(string.punctuation))


# Function to tokenize and preprocess a document
def preprocess(text):
    tokens = set(word_tokenize(text.lower())) # get all tokens
    return [i for i in tokens if i not in stop] # get all tokens without stopwords

    # add lematization later
    
def query_inverted_index(query):
    with shelve.open("../Inverted_index/inverted_index.db") as db:
        terms = [t for t in preprocess(query) if t in db]
        if not terms:
            return []
        terms.sort(key=lambda t: len(db[t]))

        result = db[terms[0]]
        for term in terms[1:]:
            result = AND(result, db[term])   
        return result
    

In [51]:
query = "sciences are great because people Mannheim"

result = query_inverted_index(query)
print(len(result))
print(result)

105
[np.int32(78), np.int32(7853), np.int32(72787), np.int32(97220), np.int32(99978), np.int32(111464), np.int32(196139), np.int32(246186), np.int32(403184), np.int32(448119), np.int32(522096), np.int32(673521), np.int32(777589), np.int32(777915), np.int32(916041), np.int32(916189), np.int32(1025520), np.int32(1093229), np.int32(1106968), np.int32(1106992), np.int32(1124224), np.int32(1158512), np.int32(1338601), np.int32(1351599), np.int32(1418426), np.int32(1647447), np.int32(1973823), np.int32(1981656), np.int32(2029003), np.int32(2097964), np.int32(2176637), np.int32(2180791), np.int32(2182218), np.int32(2248520), np.int32(2251305), np.int32(2380829), np.int32(2540332), np.int32(2542449), np.int32(2543726), np.int32(2544878), np.int32(2549309), np.int32(2552773), np.int32(2561199), np.int32(2561598), np.int32(2563846), np.int32(2564319), np.int32(2598515), np.int32(2600323), np.int32(2601041), np.int32(2602615), np.int32(2768719), np.int32(2794708), np.int32(2804553), np.int32(3279

In [ ]:
import pandas as pd
print(testdata.head())

   query_id                                              query
0      3000  I remember this certain type of cheese from wa...
1      3001  I saw only this scene, on YouTube or Vimeo a l...
2      3002  young boy works with older master to learn jud...
3      3003  This building looks like a big hotel in New Yo...
4      3004  I vividly remember strolling through the archi...


In [ ]:
import indexed_gzip

def read_lines_from_gzip(gz_path, line_numbers):
    line_numbers_set = set(line_numbers)  # faster lookup
    lines = []
    
    with indexed_gzip.open(filename=gz_path, mode='rb') as f:
        for i, line in enumerate(f, start=1):
            data = loads(line)
            id = data.get("id", "")
            if i in line_numbers_set:
                lines.append(id)
            if len(lines) == len(line_numbers_set):
                break
    return lines

# Example usage
line_numbers_to_read = [2, 5, 10]
lines = read_lines_from_gzip(corpus, line_numbers_to_read)

print(lines)

['39', '305', '324']


In [70]:
def evaluateOnJSON(filenames, runID):
    for filename in filenames:
        testdata = pd.read_json(filename, lines = True)
        testdata["result"] = [[1, 2, 3, 4] for _ in range(len(testdata))]
        for index, row in testdata.iterrows():
            result = query_inverted_index(row["query"])
            result = [1, 2, 3, 4] # remove             
            pageIDS = read_lines_from_gzip(corpus, result) 
            for i in range(len([1, 2, 3, 4])):
                print(f"{row["query_id"]} Q0 {pageIDS[i]} {i + 1} {1.0 / len(result)} {runID}")
            break
        

# runID ggfs. manuell setzen :)
evaluateOnJSON(["../data/tot25/test-2025-queries.jsonl"], "1")


3000 Q0 12 1 0.25 1
3000 Q0 39 2 0.25 1
3000 Q0 290 3 0.25 1
3000 Q0 303 4 0.25 1
